In [50]:

import os
from scipy import *
from numpy import *
from scipy.ndimage import *
import pylab
#!/usr/bin/env python
import cv2
from scipy import *
from scipy.linalg import *
from scipy.special import *
from random import choice
from PIL import Image
import sys

In [52]:
"""
Python module for use with David Lowe's SIFT code available at:
http://www.cs.ubc.ca/~lowe/keypoints/
adapted from the matlab code examples.

Initial code by Jan Erik Solem, 2009-01-30
"""



def match(desc1,desc2):
    """ 
        for each descriptor in the first image, select its match to second image    
        
        
    """

    dist_ratio = 0.6
    desc1_size = desc1.shape

    matchscores = zeros((desc1_size[0],1))
    desc2t = desc2.T #precompute matrix transpose
    for i in range(desc1_size[0]):
        dotprods = dot(desc1[i,:],desc2t) #vector of dot products
        dotprods = 0.9999*dotprods
        #inverse cosine and sort, return index for features in second image
        indx = argsort(arccos(dotprods))

        #check if nearest neighbor has angle less than dist_ratio times 2nd
        if arccos(dotprods)[indx[0]] < dist_ratio * arccos(dotprods)[indx[1]]:
            matchscores[i] = indx[0]

    return matchscores

import matplotlib.pyplot as plt

    
def appendimages(im1, im2):
    """ Return a new concatenated images side-by-side """
    if ndim(im1) == 2:
        return _appendimages(im1, im2)
    else:
        imr = _appendimages(im1[:, :, 0], im2[:, :, 0])
        img = _appendimages(im1[:, :, 1], im2[:, :, 1])
        imb = _appendimages(im1[:, :, 2], im2[:, :, 2])
        return dstack((imr, img, imb))


def _appendimages(im1,im2):
    """ return a new image that appends the two images side-by-side."""

    #select the image with the fewest rows and fill in enough empty rows
    rows1 = im1.shape[0]
    rows2 = im2.shape[0]

    if rows1 < rows2:
        im1 = concatenate((im1,zeros((rows2-rows1,im1.shape[1]))), axis=0)
    else:
        im2 = concatenate((im2,zeros((rows1-rows2,im2.shape[1]))), axis=0)

    return concatenate((im1,im2), axis=1)


In [55]:
from scipy import *
from scipy import linalg
from scipy import ndimage

'''
    Homography
'''

def Haffine_from_points(fp,tp):
    """ find H, affine transformation, such that
        tp is affine transf of fp"""

    if fp.shape != tp.shape:
        raise RuntimeError

    #condition points
    #-from points-
    m = mean(fp[:2], axis=1)
    maxstd = max(std(fp[:2], axis=1))
    C1 = diag([1/maxstd, 1/maxstd, 1])
    C1[0][2] = -m[0]/maxstd
    C1[1][2] = -m[1]/maxstd
    fp_cond = dot(C1,fp)

    #-to points-
    m = mean(tp[:2], axis=1)
    C2 = C1.copy() #must use same scaling for both point sets
    C2[0][2] = -m[0]/maxstd
    C2[1][2] = -m[1]/maxstd
    tp_cond = dot(C2,tp)

    #conditioned points have mean zero, so translation is zero
    A = concatenate((fp_cond[:2],tp_cond[:2]), axis=0)
    U,S,V = linalg.svd(A.T)

    #create B and C matrices as Hartley-Zisserman (2:nd ed) p 130.
    tmp = V[:2].T
    B = tmp[:2]
    C = tmp[2:4]

    tmp2 = concatenate((dot(C,linalg.pinv(B)),zeros((2,1))), axis=1)
    H = vstack((tmp2,[0,0,1]))

    #decondition
    H = dot(linalg.inv(C2),dot(H,C1))

    return H / H[2][2]

def affine_transform2(im, rot, shift):
    '''
        Perform affine transform for 2/3D images.
    '''
    if ndim(im) == 2:
        return ndimage.affine_transform(im, rot, shift)
    else:
        imr = ndimage.affine_transform(im[:, :, 0], rot, shift)
        img = ndimage.affine_transform(im[:, :, 1], rot, shift)
        imb = ndimage.affine_transform(im[:, :, 2], rot, shift)

        return dstack((imr, img, imb))

In [123]:



def get_points(kp1, kp2, matchscores):
    '''
        Return the corresponding points in both the images
    '''
    plist = []
    t = min(len(locs1), len(locs2))
    for i in range(len(matchscores)):
        if (matchscores[i] > 0):
            y1 = int(locs1[i, 1])
            x1 = int(locs1[i, 0])

            y2 = int(locs2[int(matchscores[i]), 1])
            x2 = int(locs2[int(matchscores[i]), 0])

            plist.append([[x1,y1],[x2,y2]])
    return plist

def get_homography(points_list):
    '''
        Function to quickly compute a homography matrix from all point 
        correspondences.

        Inputs:
            points_list: tuple of tuple of tuple of correspondence indices. Each
            entry is [[x1, y1], [x2, y2]] where [x1, y1] from image 1 corresponds
            to [x2, y2] from image 2.

        Outputs:
            H: Homography matrix.
    '''
    fp = ones((len(points_list), 3))
    tp = ones((len(points_list), 3))

    for idx in range(len(points_list)):
        fp[idx, 0] = points_list[idx][0][0]
        fp[idx, 1] = points_list[idx][0][1]

        tp[idx, 0] = points_list[idx][1][0]
        tp[idx, 1] = points_list[idx][1][1]

    H = Haffine_from_points(fp.T, tp.T)

    return H


def ransac(im1, im2, points_list, iters = 300 , error = 10, good_model_num = 15):
    '''
        This function uses RANSAC algorithm to estimate the
        shift and rotation between the two given images
    '''

    if ndim(im1) == 2:
        rows,cols = im1.shape
    else:
        rows, cols, _ = im1.shape

    model_error = 255
    model_H = None

    for i in range(iters):
        consensus_set = []
        points_list_temp = copy(points_list).tolist()
        # Randomly select 3 points
        for j in range(3):
            temp = choice(points_list_temp)
            consensus_set.append(temp)
            points_list_temp.remove(temp)

        # Calculate the homography matrix from the 3 points

        fp0 = []
        fp1 = []
        fp2 = []

        tp0 = []
        tp1 = []
        tp2 = []
        for line in consensus_set:

            fp0.append(line[0][0])
            fp1.append(line[0][1])
            fp2.append(1)

            tp0.append(line[1][0])
            tp1.append(line[1][1])
            tp2.append(1)

        fp = array([fp0, fp1, fp2])
        tp = array([tp0, tp1, tp2])

        H = Haffine_from_points(fp, tp)

        # Transform the second image
        # imtemp = transform_im(im2, [-xshift, -yshift], -theta)
        # Check if the other points fit this model

        for p in points_list_temp:
            x1, y1 = p[0]
            x2, y2 = p[1]

            A = array([x1, y1, 1]).reshape(3,1)
            B = array([x2, y2, 1]).reshape(3,1)

            out = B - dot(H, A)
            dist_err = hypot(out[0][0], out[1][0])
            if dist_err < error:
                consensus_set.append(p)


        # print(len(consensus_set))
        # Check how well is our speculated model
        if len(consensus_set) >= good_model_num:
            dists = []
            for p in consensus_set:
                x0, y0 = p[0]
                x1, y1 = p[1]

                A = array([x0, y0, 1]).reshape(3,1)
                B = array([x1, y1, 1]).reshape(3,1)

                out = B - dot(H, A)
                dist_err = hypot(out[0][0], out[1][0])
                dists.append(dist_err)
            if (max(dists) < error) and (max(dists) < model_error):
                model_error = max(dists)
                model_H = H
    
    print(model_H)
    return model_H


   

In [ ]:
def ransac_new(data, modelClass, minDataPts, nIter, threshold, nCloseRequired):
    best_model = None
    best_consensus_set = None
    best_error = np.inf
    for iteration in xrange(nIter):
        maybe_inliers = data[random.sample(range(len(data)),minDataPts)]
        maybe_model = modelClass()
        maybe_model.fit(maybe_inliers)

        fit_errors = maybe_model.calc_error(data)
        consensus_set = np.flatnonzero(fit_errors < threshold)
        
        if len(consensus_set) > nCloseRequired:
            maybe_model.fit(data[consensus_set])
            new_fit_errors = maybe_model.calc_error(data)
            total_error = new_fit_errors.sum()
            if total_error < best_error:
                best_model = maybe_model
                best_consensus_set = np.flatnonzero(new_fit_errors < threshold) # here I differ from wikipedia
                best_error = total_error

    return best_model, best_consensus_set, best_error

In [96]:
def warp_images(image0, image1, transform):
    r, c = image1.shape[:2]
    # Note that transformations take coordinates in (x, y) format,
    # not (row, column), in order to be consistent with most literature
    corners = np.array([[0, 0],
                        [0, r],
                        [c, 0],
                        [c, r]])

    # Warp the image corners to their new positions
    warped_corners = transform(corners)

    # Find the extents of both the reference image and the warped
    # target image
    all_corners = np.vstack((warped_corners, corners))

    corner_min = np.min(all_corners, axis=0)
    corner_max = np.max(all_corners, axis=0)

    output_shape = (corner_max - corner_min)
    output_shape = np.ceil(output_shape[::-1])

    offset = SimilarityTransform(translation=-corner_min)

    image0_ = warp(image0, offset.inverse, output_shape=output_shape, cval=-1)

    image1_ = warp(image1, (transform + offset).inverse, output_shape=output_shape, cval=-1)

    image0_zeros = warp(image0, offset.inverse, output_shape=output_shape, cval=0)

    image1_zeros = warp(image1, (transform + offset).inverse, output_shape=output_shape, cval=0)

    overlap = (image0_ != -1.0 ).astype(int) + (image1_ != -1.0).astype(int)
    overlap += (overlap < 1).astype(int)
    merged = (image0_zeros+image1_zeros)/overlap

    im = Image.fromarray((255*merged).astype('uint8'), mode='RGB')
    im.save('stitched_images.jpg')
    im.show()

In [96]:
img_1_left_path = "data\\part1\\left.jpg"
img_1_right_path = "data\\part1\\right.jpg"

In [78]:
from scipy.spatial.distance import cdist


def plot_matches(img1,img2,locs):
    """ show image with features. input: im (image as array),
        locs (row, col, scale, orientation of each feature) """
    
    implot = plt.imshow(img1)    
    for points in locs:
        x,y = points[0]
        plt.scatter(x,y)
    plt.show()
    
    implot2 = plt.imshow(img2)
    for points in locs:
        x,y = points[1]
        plt.scatter(x,y)
    plt.show()


In [124]:
    
img1_l = Image.open(img_1_left_path)
img1_r = Image.open(img_1_right_path)

im1_l = asarray(img1_l)
im1_r = asarray(img1_r)

sift = cv2.xfeatures2d.SIFT_create()
'''
    kp is a list of key points
    des is a numpy array of shape Number_of_Keypoints×128.
'''
kp1, des1 = sift.detectAndCompute(im1_l, None)
kp2, des2 = sift.detectAndCompute(im1_r, None)
# print("desc shape", des1)
distance = cdist(des1, des2, 'sqeuclidean')
"""
Select putative matches based on the matrix of pairwise descriptor distances obtained above. 
You can select all pairs whose descriptor distances are below a specified threshold, 
or select the top few hundred descriptor pairs with the smallest pairwise distances.
"""
# num of points to extract 
total_select_numbers = min(300, len(kp1), len(kp2))
select_match = []
INF = 1111111

for i in range(total_select_numbers):
        ri,ci = np.unravel_index(distance.argmin(), distance.shape)
        x1,y1 = kp1[ri].pt
        x1,y1 = int(x1),int(y1)
        x2,y2 = kp2[ci].pt
        x2,y2 = int(x2),int(y2)
        select_match.append([[x1,y1],[x2,y2]])
        # set to INF after used
        distance[ri, :] = INF
        distance[:, ci] = INF

print(len(select_match))
plot_matches(img1_l,img1_r,select_match)
    
   

300


In [126]:

 # Compare ransac and simple homography matrix
 
 #workable param: iterationransac(im1, im2, points_list, 
 # iters = 300 , error = 10, good_model_num = 15):

out_ransac = ransac(im1_l, im1_r, select_match,
                    iters = 300 ,
                    error=10,
                    good_model_num=10
                    )
out_simple = get_homography(select_match)

H_ransac = inv(out_ransac)
H_simple = inv(out_simple)

im_ransac = affine_transform2(im1_l,
                              H_ransac[:2, :2],
                              [H_ransac[0][2], H_ransac[1][2]])

im_simple = affine_transform2(im1_l,
                              H_simple[:2, :2],
                              [H_simple[0][2], H_simple[1][2]])
Image.fromarray(im1_r).show()
Image.fromarray(im_ransac).show()
Image.fromarray(im_simple).show()



[[ 1.90531716e-01  3.44193097e+00 -7.05756996e+02]
 [ 1.15671642e-01  6.30597015e-01  2.54253731e+01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


AttributeError: 'Stitcher' object has no attribute 'detectAndDescribe'